In [1]:
# (İstersen) ortamı temizleme vs. kısımlarını ekleyebilirsin.
!pip install -q "transformers>=4.51.3" bitsandbytes accelerate peft sentence-transformers faiss-cpu

import torch, os
device = "cuda" if torch.cuda.is_available() else "cpu"
device


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 115.1 MB/s eta 0:00:00


'cuda'

In [2]:
# Repo'yu klonla
!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

BASE_REPO = "/content/YapayZekaChatbot"
print(os.listdir(BASE_REPO))


Cloning into 'YapayZekaChatbot'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 290 (delta 9), reused 18 (delta 5), pack-reused 268 (from 2)
Receiving objects: 100% (290/290), 573.88 MiB | 17.08 MiB/s, done.
Resolving deltas: 100% (144/144), done.
['.gitattributes', 'FINE-TUNE', 'RAG', 'README.md', '.git', 'bilgi_guvenligi_dataset']


In [5]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

chunks_meta_path = os.path.join(BASE_REPO, "RAG", "chunks_meta.csv")
chunks_df = pd.read_csv(chunks_meta_path)

print("Chunk sayısı:", len(chunks_df))
chunks_df.head()


Chunk sayısı: 247


,chunk_id,filename,section_label,chunk_index,text
0,0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
1,1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.10,0,5.10 Bilginin ve diğer ilgili varlıkların kabu...
2,2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.11,0,5.11 Varlıkların İadesi Personel ve diğer ilgi...
3,3,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.12,0,"5.12 Bilgilerin Sınıflandırılması Bilgiler, ku..."
4,4,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.13,0,5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...


In [4]:
from huggingface_hub import notebook_login
notebook_login() # Karşınıza çıkan pencerede HF token'ınızı girin.

In [6]:
import torch

model_name = "intfloat/multilingual-e5-large"   # RAG defterinde kullandığın model
embed_model = SentenceTransformer(model_name, device=device)
embed_model.max_seq_length = 512

import os

INDEX_PATH     = os.path.join(BASE_REPO, "RAG", "faiss_index.bin")
CHUNK_IDS_PATH = os.path.join(BASE_REPO, "RAG", "chunk_ids.npy")

if os.path.exists(INDEX_PATH) and os.path.exists(CHUNK_IDS_PATH):
    print("✅ GitHub'daki hazır FAISS index ve chunk_ids kullanılacak")
    index = faiss.read_index(INDEX_PATH)
    chunk_ids = np.load(CHUNK_IDS_PATH)
else:
    print("⚠️ FAISS dosyaları bulunamadı, yeniden embed edip index kuruyorum (biraz zaman alır)...")

    texts = chunks_df["text"].tolist()
    chunk_ids = chunks_df["chunk_id"].to_numpy()

    passages = [f"passage: {t}" for t in texts]

    batch_size = 32
    emb_list = []

    from tqdm.auto import tqdm

    for i in tqdm(range(0, len(passages), batch_size), desc="Chunk embedding"):
        batch = passages[i:i+batch_size]
        with torch.no_grad():
            emb = embed_model.encode(
                batch,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=True
            )
        emb_list.append(emb)

    embeddings = np.vstack(emb_list)
    dim = embeddings.shape[1]

    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    print("Embedding shape:", embeddings.shape)
    print("FAISS vektör sayısı:", index.ntotal)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

✅ GitHub'daki hazır FAISS index ve chunk_ids kullanılacak


In [21]:
import pandas as pd
import os

chunks_path = os.path.join(
    BASE_REPO,
    "bilgi_guvenligi_dataset",
    "bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL.csv"
)

chunks_df = pd.read_csv(chunks_path)

# Türkçe metin kolonu
TEXT_COL = "chunk_text_tr"


In [22]:
RAG_MIN_SCORE = 0.80  # bilmediği soruya cevap vermesin diye eşik

def search_chunks(query: str, top_k: int = 5):
    query_text = f"query: {query}"

    with torch.no_grad():
        q_emb = embed_model.encode(
            [query_text],
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )

    scores, indices = index.search(q_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        # Eğer chunk_ids kullandıysan:
        cid = int(chunk_ids[idx])
        row = chunks_df.iloc[cid]   # veya chunks_df[chunks_df["chunk_index"]==cid].iloc[0]

        filename = row["filename"]
        text     = row[TEXT_COL]
        url      = row["source_url"]   # 🔥 direk buradan alıyoruz
        section_label = row["section_label"]

        results.append({
            "score": float(score),
            "chunk_id": cid,
            "chunk_index": int(row["chunk_index"]),
            "section_label": section_label,
            "filename": filename,
            "text": text,
            "url": url
        })
    return results


In [16]:
test_res = search_chunks("Uzaktan çalışma sırasında VPN neden zorunludur?", top_k=3)
test_res


[{'score': 0.8622845411300659,
  'chunk_id': 43,
  'chunk_index': 0,
  'section_label': 'ISO_6.7',
  'filename': 'ISO-27001-Controls-List-Free-Download-pdf-.pdf',
  'text': '6.7 Uzaktan çalışma Personel, kuruluşun tesisleri dışında erişilen, işlenen veya saklanan bilgileri korumak için uzaktan çalışırken güvenlik önlemleri uygulanmalıdır.\n6.7 Remote working Security measures should be implemented when personnel are working remotely to protect information accessed, processed or stored outside the organization’s premises.',
  'url': None},
 {'score': 0.8348915576934814,
  'chunk_id': 188,
  'chunk_index': 1,
  'section_label': 'Safeguards',
  'filename': 'SANS_Perimeter_Network_Access_Management_Policy_April2025.pdf',
  'text': "ağ bağlantılarının filtrelenmesi, yalnızca yetkili kullanıcıların bir kuruluşun ağına uzaktan bağlanabilmesini sağlar. PNA-09 Kuruluşun onaylı çevre ağı güvenlik duvarlarının, tüm uzak bağlantıların kimliğini doğrularken Çok Faktörlü Kimlik Doğrulaması (MFA) ger

In [11]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL_ID = "google/gemma-2-2b-it"
ADAPTER_PATH  = os.path.join(BASE_REPO, "FINE-TUNE", "gemma-2b-train-qlora-final")

# 1) Tokenizer'ı adapter klasöründen al
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, use_fast=True)

# 2) Base Gemma modelini yükle
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# 3) LoRA adapter'ı giydir
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
)

# 4) PAD/EOS hizalama
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

print("✅ Fine-tune edilmiş Gemma 2B (LoRA adaptörlü) yüklendi!")


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

✅ Fine-tune edilmiş Gemma 2B (LoRA adaptörlü) yüklendi!


In [26]:
def build_rag_user_content(question: str, retrieved_chunks) -> str:
    """
    RAG ile gelen chunk'ları kullanarak kullanıcı mesajını (context + soru) üretir.
    """
    if not retrieved_chunks:
        context_text = "İlgili kaynak bulunamadı."
    else:
        context_blocks = []
        for i, ch in enumerate(retrieved_chunks, 1):
            block = (
                f"[{i}] Kaynak (dosya: {ch['filename']}, bölüm: {ch['section_label']}, chunk_index: {ch['chunk_index']})\n"
                f"{ch['text']}"
            )
            context_blocks.append(block)
        context_text = "\n\n".join(context_blocks)

    user_content = (
        "Sen ISO 27001 ve SANS bilgi güvenliği politikaları konusunda uzman bir asistansın. "
        "Aşağıda ilgili politika parçalarını göreceksin.\n\n"
        "Sadece bu kaynaklara dayanarak cevap ver. "
        "Eğer kaynaklar soruyu yanıtlamaya yetmiyorsa veya emin değilsen:\n"
        "- 'Kaynaklarda bu sorunun yanıtı yok.' de\n"
        "- KESİNLİKLE tahmin yürütme veya uydurma.\n\n"
        "=== KAYNAK METİNLER ===\n"
        f"{context_text}\n\n"
        "=== GÖREVİN ===\n"
        "1) Soruyu bu kaynaklara dayanarak yanıtla.\n"
        "2) Kaynaklarda olmayan bilgiyi uydurma, emin değilsen bunu açıkça söyle.\n\n"
        f"=== SORU ===\n{question}"
    )

    return user_content


In [35]:
def answer_with_rag_and_ft(question: str, top_k: int = 5):
    retrieved = search_chunks(question, top_k=top_k)

    # 🔴 Kaynak yetersizse hiç cevap verme:
    if not retrieved or max(ch["score"] for ch in retrieved) < RAG_MIN_SCORE:
        links = sorted({ch["url"] for ch in retrieved if ch.get("url")})

        answer_text = (
            "Bu soruya elimdeki ISO/SANS kaynaklarıyla güvenilir bir cevap üretemiyorum. "
            "Kaynaklarda doğrudan veya yeterli düzeyde bilgi bulunmuyor."
        )

        return {
            "answer": answer_text,
            "used_chunks": retrieved,
            "source_links": links
        }

    # ✅ Yeterliyse normal RAG + fine-tune cevabı:
    user_content = build_rag_user_content(question, retrieved)

    messages = [
        {"role": "user", "content": user_content}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
      outputs = model.generate(
    **inputs,
    max_new_tokens=400,
    do_sample=False,  # ✅
)

       # outputs = model.generate(
       #     **inputs,
       #    max_new_tokens=400,
       #    do_sample=True,
       #   temperature=0.3,
       #  top_p=0.9,
       #)


    answer_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True).strip()

    links = sorted({ch["url"] for ch in retrieved if ch.get("url")})

    return {
        "answer": answer,
        "used_chunks": retrieved,
        "source_links": links
    }


In [42]:
q = "Log yönetimi neden bilgi güvenliği için zorunludur?"
result = answer_with_rag_and_ft(q, top_k=3)

print("🧠 Soru:\n", q)
print("\n--- CEVAP ---\n", result["answer"])

print("\n--- KULLANILAN KAYNAKLAR (chunk bazında) ---")
for i, k in enumerate(result["used_chunks"], 1):
    print(f"[{i}] {k['filename']} | {k['section_label']} | chunk_index={k['chunk_index']} | skor={k['score']:.3f}")

print("\n--- KAYNAK LİNKLERİ ---")
if result["source_links"]:
    for url in result["source_links"]:
        print("-", url)
else:
    print("Uygun URL bilgisi bulunamadı (csv'de url kolonu olmayabilir).")


🧠 Soru:
 Log yönetimi neden bilgi güvenliği için zorunludur?

--- CEVAP ---
 Loglar olayların izlenmesi, analiz edilmesi ve tehditlere karşı erken uyarı oluşturulması için gereklidir.

--- KULLANILAN KAYNAKLAR (chunk bazında) ---
[1] SANS_Log_Management_Policy_April2025.pdf | Safeguards | chunk_index=1 | skor=0.863
[2] SANS_Log_Management_Policy_April2025.pdf | Safeguards | chunk_index=0 | skor=0.852
[3] SANS_Log_Management_Policy_April2025.pdf | Safeguards | chunk_index=2 | skor=0.851

--- KAYNAK LİNKLERİ ---
- https://www.sans.org/information-security-policy/log-management-policy/
